In [56]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import datetime as dt
import requests
import json

In [47]:
now = dt.datetime.now()
date_today = str(now.day)+'-'+str(now.month)+'-'+str(now.year)
date_start = '02-01-2019'

# In case of scraping the web

Fuente: https://www.ambito.com/contenidos/dolar-informal-historico.html

In [11]:
# For Reference: https://www.dolarsi.com/api/api.php?type=valoresprincipales
base_html = 'H://My Drive//Formacion//Data Science//Financial//FinancialPF//Web Sites//Informal Histórico.html'
blue_hist = pd.read_html(base_html) # this parses all the tables in webpages to a list
df_blue = blue_hist[0]
df_blue.head() # Values are int without decimals, which is incorrect due to the real value's nature

,Fecha,Compra,Venta
0,05-11-2021,19500,19900
1,04-11-2021,19500,19900
2,03-11-2021,19500,19900
3,02-11-2021,19350,19750
4,01-11-2021,19250,19650


In [12]:
def decimals(df, column='Value'):
    '''Converts integer value into float considering last 2 numbers as decimals. df parameter to specify df, column parameter to specify target column of df'''
    new_value_list = []
    for value in df[column]:
        ss1, ss2 = str(value)[:-2], str(value)[-2:] # Slicing the value as string to get separated the number
        new = ss1+'.'+ss2 # Adding the . into the value
        new_value_list.append(float(new)) # Adding the new value to a list}
    df[column] = pd.Series(new_value_list) # Converting it to a Series and inserting replacing the extracted values

In [13]:
df_blue.columns = ['Date', 'Buy', 'Sell']

In [14]:
decimals(df_blue, 'Buy')
decimals(df_blue, 'Sell')

In [15]:
df_blue

,Date,Buy,Sell
0,05-11-2021,195.00,199.00
1,04-11-2021,195.00,199.00
2,03-11-2021,195.00,199.00
3,02-11-2021,193.50,197.50
4,01-11-2021,192.50,196.50
...,...,...,...
718,08-01-2019,37.25,39.25
719,07-01-2019,37.50,39.50
720,04-01-2019,38.00,40.00
721,03-01-2019,38.25,40.25


In [16]:
df_blue.to_csv('H:\My Drive\Formacion\Data Science\Financial\FinancialPF\CSVs\Blue.csv', index=False)

# In case of using the web's API

In [48]:
# API Ámbito
request_url = 'https://mercados.ambito.com//dolarturista/historico-general/' #20-10-2021/20-11-2021'
request_url = request_url+date_start+'/'+date_today

In [49]:
response = requests.get(request_url)

In [77]:
response = response.text[1:]
response = response[:-1]

In [93]:
response = response.replace('],[', '];[')

In [103]:
(response).split(';')[10].replace('","', '";"') # Identifying tuples

'["08-11-2021";"174,21"]'

In [134]:
(response).split(';')[10].replace('","', '";"').replace('[','').replace(']','').replace('"','').split(';') # Splitting values

['08-11-2021', '174,21']

In [128]:
pd.to_datetime(date_today)-pd.to_datetime(date_start)

Timedelta('1024 days 00:00:00')

In [149]:
prices = []

In [150]:
for i in range (0,1025):
    try:
        prices.append(((response).split(';')[i].replace('","', '";"').replace('[','').replace(']','').replace('"','').split(';')))
    except:
        pass

In [152]:
Blue = pd.DataFrame(prices[1:], columns=prices[0])
Blue

,Fecha,Venta
0,19-11-2021,"174,83"
1,18-11-2021,"174,82"
2,17-11-2021,"174,80"
3,16-11-2021,"174,77"
4,15-11-2021,"174,70"
...,...,...
476,02-01-2020,"81,90"
477,30-12-2019,"81,89"
478,27-12-2019,"81,89"
479,26-12-2019,"81,89"


In [154]:
path = 'H:\My Drive\Formacion\Data Science\Financial\FinancialPF\CSVs\\'
Blue.to_csv(path+'Blue_API.csv', index=False)